In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
np.random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
dataset_train = torchvision.datasets.MNIST(root='', train=True, transform=transforms.ToTensor())
dataset_test = torchvision.datasets.MNIST(root='', train=False, transform=transforms.ToTensor())

In [4]:
# parameters
lr = 0.001
training_epochs = 15
batch_size = 64
dropout_rate = 0.5

In [5]:
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, drop_last=True)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True, drop_last=True)

In [6]:
class CNNModel(nn.Module):

    def __init__(self):
        '''
        L1 ImgIn shape=(?, 28, 28, 1)
           Conv     -> (?, 28, 28, 32)
           Pool     -> (?, 14, 14, 32)

        L2 ImgIn shape=(?, 14, 14, 32)
           Conv      ->(?, 14, 14, 64)
           Pool      ->(?, 7, 7, 64)
        
        L3 ImgIn shape=(?, 7, 7, 64)
           Conv      ->(?, 7, 7, 128)
           Pool      ->(?, 4, 4, 128)
           Flatten   ->(?, 4x4x128)

        L4 ImgIn shape=(?, 4x4x128)
           FC        ->(?, 625)
        
        L5 Final 
           FC 625 inputs -> 10 outputs
        '''
        super(CNNModel, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = torch.nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = torch.nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
            nn.Flatten())
        
        self.fc1 = nn.Linear(4 * 4 * 128, 625)
        nn.init.xavier_uniform_(self.fc1.weight)
        
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=dropout_rate))

        self.fc2 = torch.nn.Linear(625, 10)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.fc2(out)

        return out

In [7]:
model = CNNModel().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

print('Start training...')

for epoch in range(1, training_epochs+1):

    running_loss = 0
    running_correct = 0

    for X, y in dataloader_train:
        X, y = X.to(device), y.to(device)
        y_pred = model(X)
        loss = criterion(y_pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            running_loss += loss.item()
            running_correct += sum(y_pred.argmax(dim=1) == y)/batch_size

    running_test_loss = 0
    running_test_correct = 0
    
    with torch.no_grad():
        for X, y in dataloader_test:
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            loss = criterion(y_pred, y)

            running_test_loss += loss.item()
            running_test_correct += sum(y_pred.argmax(dim=1) == y)/batch_size
    
    print(f'epoch {epoch}\
    training loss:{running_loss/len(dataloader_train):.3f}\
    training acc:{running_correct/len(dataloader_train):.3f}\
    test loss:{running_test_loss/len(dataloader_test):.3f}\
    test acc:{running_test_correct/len(dataloader_test):.3f}')

print('Finished Training')

Start training...
epoch 1    training loss:0.172    training acc:0.946    test loss:0.054    test acc:0.982
epoch 2    training loss:0.050    training acc:0.985    test loss:0.048    test acc:0.984
epoch 3    training loss:0.037    training acc:0.989    test loss:0.040    test acc:0.988
epoch 4    training loss:0.029    training acc:0.991    test loss:0.036    test acc:0.990
epoch 5    training loss:0.024    training acc:0.992    test loss:0.031    test acc:0.990
epoch 6    training loss:0.020    training acc:0.994    test loss:0.029    test acc:0.991
epoch 7    training loss:0.017    training acc:0.995    test loss:0.034    test acc:0.990
epoch 8    training loss:0.015    training acc:0.995    test loss:0.038    test acc:0.990
epoch 9    training loss:0.013    training acc:0.996    test loss:0.037    test acc:0.990
epoch 10    training loss:0.010    training acc:0.997    test loss:0.033    test acc:0.991
epoch 11    training loss:0.009    training acc:0.997    test loss:0.039    test 